In [2]:
import pandas as pd
import ETL_functions as etf
import ast

# ETL

en este archivo haare todo el procesamiento de los archivos que se me dieron para generar archivos procesados de forma basica y archivos especificos para cada consulta necesaria

primero trabajare para desanidar y obtener unos archivos que me sean utiles para generar los archivos para cada consulta a partir de :

* user_reviews.json
* user_items.json

al archivo steam_games.json lo trabajare especificamente al momento de crear los dataframe finales para cada funcion

<hr>

# ETL  user_reviews.json

    cargo el archivo

In [48]:
reviews = []

with open(r"../Datasets/user_reviews.json/australian_user_reviews.json", encoding='utf-8') as f:
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))
df_ur = pd.DataFrame(reviews)

    transformo las siguiente columna a tipo str ya que es dificil manejar listas y tira errores en bastantes metodos de pandas

In [ ]:
df_ur["reviews"] = df_ur["reviews"].astype(str)

In [ ]:
etf.data_review(df_ur)

## valores nulos

    hay listas vacias que pasamos a str y por lo tanto no cuentan como nulos pero igualmente tendremos que borrar despues

In [ ]:
mask = df_ur["reviews"] == "[]"

In [ ]:
df_ur[mask]

    estas son todas las reviews con columna reviews nulos no las borrar ya que estaria borrando la fila y no quiero perder los registros de usuarios que podrian servir para normalizar otros archivos

de este archivo pienso sacar 2 ya que no es nesesario saber quien hizo la review puedo separarlo en 2 dataframe distintos :

uno con la parte de  informacion de el usuario que puede usarse para normalizar otras columnas

otro con las reviews desaanidas y el analisis de sentimiento

    para crear el primer archivo voy a dropear la columna reviews

In [51]:
del df_ur["reviews"]

In [52]:
etf.data_review(df_ur)


Total rows:  25799

Total full null rows:  0

Total duplicated rows: 314


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,user_id,[<class 'str'>],100.0,25799,0.0,0
1,user_url,[<class 'str'>],100.0,25799,0.0,0


    vemos que tenemos filas duplicadas  las  que tendremos que borrar

In [53]:
df_ur.drop_duplicates(inplace=True)

## dataframe resultante
|columna |tipo|
|--------|----|
|user_id | str|
|user_url| str|

## guardamos en un archivo parquet

In [55]:
df_ur.to_parquet(r"../Datasets/user_data.parquet")#dentro de la carpeta datasets

    para el segundo archivo tendremos que volver a importar el archivo a un dataaframe pero esta vez conservar solo la columna reviews

In [56]:
reviews = []

with open(r"../Datasets/user_reviews.json/australian_user_reviews.json", encoding='utf-8') as f:
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))
df_ur = pd.DataFrame(reviews)

    probando como se comporta la columna anidada frente al metodo de pandas pd.json_normalize

In [57]:
pd.json_normalize(df_ur["reviews"])#al hacer json_normalize nos  devuelve de esta forma un df de diccionarios

,0,1,2,3,4,5,6,7,8,9
0,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
25794,"{'funny': '', 'posted': 'Posted May 31.', 'las...",None,None,None,None,None,None,None,None,None
25795,"{'funny': '', 'posted': 'Posted June 17.', 'la...",None,None,None,None,None,None,None,None,None
25796,"{'funny': '1 person found this review funny', ...",None,None,None,None,None,None,None,None,None
25797,"{'funny': '', 'posted': 'Posted July 21.', 'la...","{'funny': '', 'posted': 'Posted July 10.', 'la...","{'funny': '', 'posted': 'Posted July 10.', 'la...","{'funny': '', 'posted': 'Posted July 8.', 'las...",None,None,None,None,None,None


In [58]:
pd.json_normalize(df_ur["reviews"][0])# al hacer json_normalize en cada fila nos devuelve el siguiente dataframe que es lo que me interesa

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


    basado en este comportamiento me di cuenta de que lo puedo maanejar a travez de iteraciones y json_normaalizer cada que necesite acceder a sus valores lo que me creara un dataframe dentro de un bucle con la columna que yo le pida metodo que voy a utlilizar en el user_items dado  su gran tamaño y en este caso al ser mucho mas ligero puedo permitirme crear un dataframe a partir de todos estos datos

### bucle para obtener todos los diccionarios en un dataframe llamaado df_sentimientos

    al hacer esto perdemos quien hizo la review (dato que no nos es necesario) y conservamos el id de el juego 

In [59]:
df_sentimientos = pd.DataFrame(columns=["funny","posted","last_edited","item_id","helpful","recommend","review"])

for i in df_ur["reviews"]:
    var = pd.json_normalize(i)
    # Supongamos que df1 y df2 son tus DataFrames con las mismas columnas
    df_sentimientos = pd.concat([df_sentimientos, var], axis=0)

df_sentimientos

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
0,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...
2,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
3,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
0,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
1,,Posted July 20.,,730,No ratings yet,True,:D


solo conservaremos las  columnas:

* posted(para tener referencia a una fecha)
* item_id (para  relacionar con otros dataframes)
* recommend(para saber si se recomienda o no un juego)
* review (columna que usaremos para el aanalisis de sentimiento)


In [60]:
del df_sentimientos["funny"]
del df_sentimientos["helpful"]
del df_sentimientos["last_edited"]

    mostramos el archivo sin estas columnas que no utilizaremos

In [61]:
df_sentimientos

,posted,item_id,recommend,review
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
0,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
1,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...
2,Posted July 10.,70,True,a must have classic from steam definitely wort...
3,Posted July 8.,362890,True,this game is a perfect remake of the original ...
0,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
1,Posted July 20.,730,True,:D


    asi nos quedaria el dataframe que usaremos para analisis  de sentimientos posted se podria borrar pero pienso dejarla por el momento

In [62]:
etf.data_review(df_sentimientos)


Total rows:  59305

Total full null rows:  0

Total duplicated rows: 881


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,posted,[<class 'str'>],100.0,59305,0.0,0
1,item_id,[<class 'str'>],100.0,59305,0.0,0
2,recommend,[<class 'bool'>],100.0,59305,0.0,0
3,review,[<class 'str'>],100.0,59305,0.0,0


    borro las filas duplicadas

In [67]:
df_sentimientos.drop_duplicates(inplace=True)

In [68]:
df_sentimientos.duplicated().sum()#ya no contamos con duplicados

0

## guardar el archivo como parquet

In [70]:
df_sentimientos.to_parquet(r"../Datasets/reviews_dict.parquet")

<hr>

#  ETL user_items.json

## cargamos el archivo

In [8]:
items = []

with open(r'../Datasets/users_items.json/australian_users_items.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        items.append(ast.literal_eval(line))

df_ui = pd.DataFrame(items)

## usamos dos bucles para desanidar la lista de json anidados y guardar los valores en nuevas columnas

In [9]:
desanidado = []
for index, row in df_ui.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    user_items_count = row['items_count']
    for df_items_dict in row['items']:
        df_items_dict['user_id'] = user_id
        df_items_dict['user_url'] = user_url
        df_items_dict['items_count'] = user_items_count
        desanidado.append(df_items_dict)

df_ui = pd.DataFrame(desanidado)

    estes el dataframe resultante de la ejecucion de los bucles

In [10]:
df_ui

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,user_url,items_count
0,10,Counter-Strike,6,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
1,20,Team Fortress Classic,0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
2,30,Day of Defeat,7,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
3,40,Deathmatch Classic,0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
4,50,Half-Life: Opposing Force,0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7
5153205,373330,All Is Dust,0,0,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7


In [12]:
etf.data_review(df_ui)


Total rows:  5153209

Total full null rows:  0

Total duplicated rows: 59104


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,item_id,[<class 'str'>],100.0,5153209,0.0,0
1,item_name,[<class 'str'>],100.0,5153209,0.0,0
2,playtime_forever,[<class 'int'>],100.0,5153209,0.0,0
3,playtime_2weeks,[<class 'int'>],100.0,5153209,0.0,0
4,user_id,[<class 'str'>],100.0,5153209,0.0,0
5,user_url,[<class 'str'>],100.0,5153209,0.0,0
6,items_count,[<class 'int'>],100.0,5153209,0.0,0


    podemos notar que tenemos filas duplicadas

In [13]:
df_ui.drop_duplicates(inplace=True)#borramos las filas duplicadas

    ahora si revisamos ya no tenemos duplicados

In [14]:
df_ui.duplicated().sum()

0

In [16]:
df_ui.shape#estas son las filas sin de el dataframe sin duplicados y las columnas

(5094105, 7)

In [37]:
df_ui.sample(7)

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,user_url,items_count
183091,282440,Quake Live,0,0,76561198141880878,http://steamcommunity.com/profiles/76561198141...,55
2317699,730,Counter-Strike: Global Offensive,26264,0,weoweoman,http://steamcommunity.com/id/weoweoman,209
4369819,218230,PlanetSide 2,900,0,76561198046110507,http://steamcommunity.com/profiles/76561198046...,41
2660037,65930,The Bureau: XCOM Declassified,7,0,ImJustPro,http://steamcommunity.com/id/ImJustPro,247
3632741,265670,Imagine Me,83,0,TuDragon76a,http://steamcommunity.com/id/TuDragon76a,1269
256698,203160,Tomb Raider,852,0,76561197970812298,http://steamcommunity.com/profiles/76561197970...,149
4298878,3620,Zuma's Revenge,0,0,76561198042684717,http://steamcommunity.com/profiles/76561198042...,147


    podemos decir que playtime_2weeks no la vamos a usar teniendo el total de tiempo jugado(playtime_forever) 

In [38]:
del df_ui["playtime_2weeks"]#borramos la columna 

In [39]:
df_ui.sample(3)

,item_id,item_name,playtime_forever,user_id,user_url,items_count
613858,377160,Fallout 4,2325,76561198055528217,http://steamcommunity.com/profiles/76561198055...,69
3369360,385200,Absconding Zatwor,408,-SEVEN-,http://steamcommunity.com/id/-SEVEN-,394
2223941,730,Counter-Strike: Global Offensive,210,thesexyjesus,http://steamcommunity.com/id/thesexyjesus,77


    tambien borrare la columna user_url ya que voy a usar user_id para referirme al usuario

In [40]:
del df_ui["user_url"]

In [41]:
df_ui.sample(7)

,item_id,item_name,playtime_forever,user_id,items_count
3841412,218620,PAYDAY 2,30,76561197965326033,163
3251625,298600,Lovely Planet,0,CyclonicCyance,83
3613646,244630,NEOTOKYO°,11,SouseisekiChan,390
4883120,230410,Warframe,855,almondkung,69
360320,301970,Screencheat,160,preavus,600
530876,253710,theHunter,74,4mrtm,127
2137084,209870,Blacklight: Retribution,0,badassadmin,114


    con esto reducio bastante el archivo a las columnas que se utilizaran

In [43]:
etf.data_review(df_ui)#llamamos a esta funcion para lograr ver los tipos de datos


Total rows:  5094105

Total full null rows:  0

Total duplicated rows: 13


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,item_id,[<class 'str'>],100.0,5094105,0.0,0
1,item_name,[<class 'str'>],100.0,5094105,0.0,0
2,playtime_forever,[<class 'int'>],100.0,5094105,0.0,0
3,user_id,[<class 'str'>],100.0,5094105,0.0,0
4,items_count,[<class 'int'>],100.0,5094105,0.0,0


tabla para que se vean mejor los tipos 

|columna|tipo|
|-------|----|
|item_id|str|
|item_name|str|
|playtime_forever|int|
|user_id|str|
|items_count|int|

* item_id (numero que identifica al juego nos sirve para relacionar )
* item_name (nombre que identifica al juego nos sirve para relacionar) 
* playtime_forever (tiempo en minutos jugado por un usuario)
* user_id (identificador de usuario puede ser texto o  numerico)


items_count lo dejamos en caso de que lo necesitemos
* items_count (cantidad de items que tiene un usuario nos va a servir para remover filas)#ya no poseemos items count 0 ya que no poseemos valores nulos

    item id lo dejare como texto para luego relacionarlo asi que abajo verificare  si todos sus valores "texto" son numericos con una funcion

In [42]:
etf.valores_no_numericos(df_ui["item_id"])#devuelve una lista con los valores no numericos que se encuentren en una columna

[]

    en este caso devolvio una lista vacia lo que significa que no hay valores que no sean numeros en item_id

## guardar el archivo en parquet

In [47]:
df_ui.to_parquet(r"../Datasets/df_uitems.parquet")

<hr>

# analisis de sentimiento

    volvemos a cargar el archivo dict reviews que creamos mas arriba para agregarle la columna de analisis de sentimiento que nos servira para varias funciones

In [73]:
from textblob import TextBlob

In [72]:
df_ur = pd.read_parquet(r"../Datasets/reviews_dict.parquet")

los valores que tomara esta nueva columna seran los siguientes:

|valor|representa|
|-----|----------|
|0    |negativo  |
|1    |neutral   |
|2    |positivo  |

    si un review no tiene texto en la parte de comentario sera considerado neutral

In [74]:
# funcion lambda que crea una columna "sentimiento" que en caso de que la review este vacia le setea valor 0 si no guarda en esta columna el valor de la polaridad
df_ur['sentimiento'] = df_ur['review'].apply(lambda x: 1 if x == "" else TextBlob(x).sentiment.polarity)

# 0 negativo si la polaridad es < a 0 , 1 si la polaridad es == 0 de otro modo significa que polaridad es mayor a 0 le damos el valor de 2 para positivo
df_ur['sentimiento'] = df_ur['sentimiento'].apply(lambda x: 0 if x < 0 else (1 if x == 0 else 2))

    con esto ya tenemos la columna sentimiento con los valores que establecimos en la tabla superior

In [75]:
df_ur

,posted,item_id,recommend,review,sentimiento
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,2
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,2
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,2
0,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,2
1,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,0
...,...,...,...,...,...
2,Posted July 10.,70,True,a must have classic from steam definitely wort...,2
3,Posted July 8.,362890,True,this game is a perfect remake of the original ...,2
0,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,2
1,Posted July 20.,730,True,:D,2


    y dropeare review ya que ya no lo necesitamos

In [76]:
del df_ur["review"]

    ya contamos con un dataframe de pocas columnas y con los valores que necesitamos asi que sobrescribire el primer archivo por este con la columna sentimiento

In [77]:
df_ur

,posted,item_id,recommend,sentimiento
0,"Posted November 5, 2011.",1250,True,2
1,"Posted July 15, 2011.",22200,True,2
2,"Posted April 21, 2011.",43110,True,2
0,"Posted June 24, 2014.",251610,True,2
1,"Posted September 8, 2013.",227300,True,0
...,...,...,...,...
2,Posted July 10.,70,True,2
3,Posted July 8.,362890,True,2
0,Posted July 3.,273110,True,2
1,Posted July 20.,730,True,2


## guardar en archivo parquet

In [ ]:
df_ur.to_parquet(r"../Datasets/reviews_dict.parquet")

<hr>

# archivos especificos

### creando el archivo df_sentimientos.parquet

este archivo sera el archivo con el que la api trabajara estas funciones:

"UsersRecommend"

es una funcion que recibe un año y devuelve el top 3 juegos más recomendados por usuarios para el año dado.(teniendo en cuenta las cantidades de recommendaciones(True) y comentarios positivos y neutrales(0 y 2))

    #Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

*UsersWorstDeveloper"

la cual va recibir año y devuelve el top 3 desarrolladoras con juegos menos recomendados para ese año (teniendo en cuenta el total de recommend igual a false y reseñas negativas )

    #Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

cargamos los archivos que utilizaremos para generar el archivo df_recommend.parquet

In [3]:
df_umr2_1 = pd.read_parquet(r"../Datasets/reviews_dict.parquet")
df_umr2_2 = pd.read_json(r"../Datasets/steam_games.json/output_steam_games.json" ,lines=True)

In [4]:
df_umr2_1

,posted,item_id,recommend,sentimiento
0,"Posted November 5, 2011.",1250,True,2
1,"Posted July 15, 2011.",22200,True,2
2,"Posted April 21, 2011.",43110,True,2
0,"Posted June 24, 2014.",251610,True,2
1,"Posted September 8, 2013.",227300,True,0
...,...,...,...,...
2,Posted July 10.,70,True,2
3,Posted July 8.,362890,True,2
0,Posted July 3.,273110,True,2
1,Posted July 20.,730,True,2


cambio  item_id a float para relacionarlo con id ya que id se encuentra en float y no me permite cambiarlo a si que para no complicarse tanto

In [16]:
df_umr2_1["item_id"] = df_umr2_1["item_id"].astype(float)

In [17]:
df_umr2_1

,posted,item_id,recommend,sentimiento
0,"Posted November 5, 2011.",1250.0,True,2
1,"Posted July 15, 2011.",22200.0,True,2
2,"Posted April 21, 2011.",43110.0,True,2
0,"Posted June 24, 2014.",251610.0,True,2
1,"Posted September 8, 2013.",227300.0,True,0
...,...,...,...,...
2,Posted July 10.,70.0,True,2
3,Posted July 8.,362890.0,True,2
0,Posted July 3.,273110.0,True,2
1,Posted July 20.,730.0,True,2


In [5]:
df_umr2_2

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [6]:
del df_umr2_2["genres"]
del df_umr2_2["price"]
del df_umr2_2["release_date"]
del df_umr2_2["reviews_url"]
del df_umr2_2["url"]
del  df_umr2_2["early_access"]
del  df_umr2_2["specs"]
del  df_umr2_2["tags"]
del  df_umr2_2["title"]
del df_umr2_2["publisher"]

In [9]:
df_umr2_2

,app_name,id,developer
0,None,NaN,None
1,None,NaN,None
2,None,NaN,None
3,None,NaN,None
4,None,NaN,None
...,...,...,...
120440,Colony On Mars,773640.0,"Nikita ""Ghost_RUS"""
120441,LOGistICAL: South Africa,733530.0,Sacada
120442,Russian Roads,610660.0,Laush Dmitriy Sergeevich
120443,EXIT 2 - Directions,658870.0,"xropi,stev3ns"


In [10]:
df_umr2_2.drop_duplicates(inplace=True)

In [11]:
df_umr2_2.head(3)

,app_name,id,developer
0,None,NaN,None
88310,Lost Summoner Kitty,761140.0,Kotoshiro
88311,Ironbound,643980.0,Secret Level SRL


In [12]:
df_umr2_2.drop(0,inplace=True)#borramos la unica columna que no poseia datos que quedaba

In [13]:
df_umr2_2.head(3)

,app_name,id,developer
88310,Lost Summoner Kitty,761140.0,Kotoshiro
88311,Ironbound,643980.0,Secret Level SRL
88312,Real Pool 3D - Poolians,670290.0,Poolians.com


In [18]:
merge = df_umr2_1.merge(df_umr2_2,how="inner",left_on="item_id",right_on="id")

In [19]:
merge

,posted,item_id,recommend,sentimiento,app_name,id,developer
0,"Posted November 5, 2011.",1250.0,True,2,Killing Floor,1250.0,Tripwire Interactive
1,"Posted March 30, 2015.",1250.0,True,2,Killing Floor,1250.0,Tripwire Interactive
2,"Posted July 12, 2013.",1250.0,True,2,Killing Floor,1250.0,Tripwire Interactive
3,"Posted August 13, 2015.",1250.0,True,1,Killing Floor,1250.0,Tripwire Interactive
4,"Posted April 5, 2014.",1250.0,True,1,Killing Floor,1250.0,Tripwire Interactive
...,...,...,...,...,...,...,...
53185,"Posted October 29, 2015.",307130.0,True,2,Asteria,307130.0,Legend Studio
53186,Posted May 5.,209120.0,True,0,Street Fighter X Tekken,209120.0,"Capcom U.S.A., Inc."
53187,Posted August 13.,220090.0,True,0,The Journey Down: Chapter One,220090.0,SkyGoblin
53188,Posted August 13.,262850.0,True,0,The Journey Down: Chapter Two,262850.0,SkyGoblin


In [21]:
etf.data_review(merge)


Total rows:  53190

Total full null rows:  0

Total duplicated rows: 8025


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,posted,[<class 'str'>],100.00,53190,0.00,0
1,item_id,[<class 'float'>],100.00,53190,0.00,0
2,recommend,[<class 'bool'>],100.00,53190,0.00,0
3,sentimiento,[<class 'int'>],100.00,53190,0.00,0
4,app_name,[<class 'str'>],100.00,53190,0.00,0
5,id,[<class 'float'>],100.00,53190,0.00,0
6,developer,"[<class 'str'>, <class 'NoneType'>]",93.29,49619,6.71,3571


    en este caso no borramos las columnas duplicadas ya que al no tener la columna revies es muy probable que haiga filas repetidas ya que son muy pocas columnas y al ya haber borrado los duplicados ,estos serian registros que de tener el comentario(review) serian distintos

lo guardamos como df_recommend.parquet

In [20]:
merge.to_parquet(r"../app/df_recommend.parquet")

<hr>